In [70]:
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [71]:
from geopy.geocoders import Nominatim
from time import sleep

In [72]:
os.getcwd()

'f:\\Clark_Universiy\\Clark_Teaching\\2025Spring\\geog386_python\\docs\\Labs\\Lab_Practice06'

In [73]:
df = pd.read_csv('mortgage.csv')
df.head()

,City,State,Mortgage,Lat,Lon
0,Urban Honolulu,HI,129877,23.7186,-164.737
1,Kapaa,HI,8786,22.0111,-159.705
2,Kahului-Wailuku-Lahaina,HI,23537,20.8677,-156.617
3,Hilo,HI,29377,19.6024,-155.523
4,Anchorage,AK,67629,62.2249,-149.553


In [74]:
# Step 1: Create population quartile groups
df['quartile'], bins = pd.qcut(df['Mortgage'], q=4, labels=["Small", "Medium", "Large", "Very Large"], retbins=True)

# Step 2: Define colors based on quartiles
quartile_colors = {
    "Small": "royalblue",
    "Medium": "crimson",
    "Large": "lightseagreen",
    "Very Large": "orange"
}

# Step 3: Update hover text
df['text'] = df['City'] + '<br>Mortgage: ' + (df['Mortgage'] / 1e6).astype(str) + ' million' + '<br>Category: ' + df['quartile'].astype(str)

fig = go.Figure()

# Step 4: Loop through quartiles instead of fixed limits
for quartile in df['quartile'].unique():
    df_sub = df[df['quartile'] == quartile]
    fig.add_trace(go.Scattergeo(
        lon=df_sub['Lon'],
        lat=df_sub['Lat'],
        text=df_sub['text'],
        marker=dict(
            size=df_sub['Mortgage'] / 2000,  # Scale marker size by population in millions
            color=quartile_colors[quartile],  # Assign colors based on quartile
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode='area'
        ),
        name=f'{quartile} Mortgage'  # Legend name for quartile
    ))

fig.update_layout(
    title_text='Housing Mortgages Across the U.S (Click legend to toggle)',
    showlegend=True,
    geo=dict(
        scope='usa',
        landcolor='rgb(217, 217, 217)',
    )
)

fig.show()


In [75]:
import pandas as pd
import plotly.graph_objects as go

# Step 1: Create population quartile groups
df['quartile'], bins = pd.qcut(df['Mortgage'], q=4, labels=["Small", "Medium", "Large", "Very Large"], retbins=True)

# Step 2: Define colors based on quartiles
quartile_colors = {
    "Small": "royalblue",
    "Medium": "crimson",
    "Large": "lightseagreen",
    "Very Large": "orange"
}

# Step 3: Update hover text
df['text'] = df['City'] + ', ' + df['State'] + '<br>Mortgage: ' + (df['Mortgage'] / 1e6).astype(str) + ' million' + '<br>Category: ' + df['quartile'].astype(str)

def create_figure():
    fig_mortgage = go.Figure()

    traces = []
    state_visibility = {}

    # Step 4: Loop through states and quartiles
    for state in df['State'].unique():
        state_visibility[state] = []
        df_state = df[df['State'] == state]
        
        for quartile in df_state['quartile'].unique():
        # for quartile in ['Very Large', 'Large', 'Medium', 'Small']:
            df_sub = df_state[df_state['quartile'] == quartile]
            trace = go.Scattergeo(
                lon=df_sub['Lon'],
                lat=df_sub['Lat'],
                text=df_sub['text'],
                marker=dict(
                    size=df_sub['Mortgage'] / 2000,  # Scale marker size
                    color=quartile_colors[quartile],
                    line_color='rgb(40,40,40)',
                    line_width=0.5,
                    sizemode='area'
                ),
                name=f'{state} - {quartile} Mortgage',
                visible=True  # All traces visible initially
            )
            traces.append(trace)
            state_visibility[state].append(len(traces) - 1)

    # Add all traces to the figure
    for trace in traces:
        fig_mortgage.add_trace(trace)

    # Create dropdown menu for filtering by state
    buttons = [
        {
            "label": "All States",
            "method": "update",
            "args": [{"visible": [True] * len(traces)}]  # Show all traces
        }
    ]

    for state, trace_indices in state_visibility.items():
        visible_array = [False] * len(traces)  # Hide all traces
        for index in trace_indices:
            visible_array[index] = True  # Only show traces of selected state
        
        buttons.append({
            "label": state,
            "method": "update",
            "args": [{"visible": visible_array}]
        })

    fig_mortgage.update_layout(
        title_text='Housing Mortgages Across the U.S (Click legend to toggle)',
        showlegend=True,
        geo=dict(
            scope='usa',
            landcolor='rgb(217, 217, 217)',
        ),
        updatemenus=[{
            "buttons": buttons,
            "direction": "down",
            "showactive": True,
        }]
    )

    return fig_mortgage

# Show figure
fig_mortgage = create_figure()
fig_mortgage.show()
